In [34]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoConfig, TFTrainer, TFTrainingArguments
from transformers.models.distilbert import TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from datasets import Dataset

In [35]:
df_train = pd.read_csv('intent_classifier_dataset.csv')
df_train


,text,intent
0,"I have people close to me willing to help, but...",report_emotion
1,I'm tired of it all and it tears me apart that...,report_emotion
2,The emptiness of losing someone you love? The ...,report_emotion
3,Hi. Just wanted to say I'm glad I found this s...,report_emotion
4,"Idk if this is the right subreddit for this, s...",report_emotion
...,...,...
495,what questions do you respond to,smalltalk
496,ai how are you doing today,greeting
497,see you around,goodbye
498,peace out,goodbye


In [36]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_train['text'].values,
    df_train['intent'].values,
    test_size=0.2,
    random_state=42
)

In [37]:
label2id = {label: idx for idx, label in enumerate(np.unique(df_train['intent']))}
id2label = {idx: label for label, idx in label2id.items()}

train_labels = [label2id[label] for label in train_labels]
val_labels = [label2id[label] for label in val_labels]

In [38]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(
    model_name,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

c:\Users\gdaob\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [42]:
# Then create model with config
model = TFDistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label2id)
)

c:\Users\gdaob\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TypeError: 'NoneType' object is not callable

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [9]:
train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [10]:
# Define compute metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "classification_report": classification_report(labels, predictions, target_names=list(label2id.keys()))
    }

In [13]:
# Training arguments
training_args = TFTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train the model
trainer.train()

# Save the model
model.save_pretrained("./intent_classifier")
tokenizer.save_pretrained("./intent_classifier")

In [ ]:
# Print final evaluation
results = trainer.evaluate()
print(results) 